# Pirate Scrabble

**This is a single-player prototype of a multiplayer game.**

**Press ENTER to flip over a tile and add it to the pool. When you spot a a word, type it and press enter to claim it.**

**You can also steal words: type a new word that contains all of the letters from a word or words you want to steal, and optionally letters from the pool as well.**

----
  
  
  

In [1]:
from IPython.display import clear_output, display
import random
import string
import ipywidgets as widgets

with open('twl06.txt') as twl06:    
    dictionary = [word[:-1].upper() for word in twl06.readlines()[2:]]

min_word_length = 3
alphabet = string.ascii_uppercase
pool = [13,5,6,7,24,6,7,6,12,2,2,8,8,11,15,4,2,12,10,10,6,2,4,2,2,2]
pool_flipped = [0 for i in range(26)]
played_words = []

In [25]:
def pooltoletters(letterpool):
    letterlist = [ alphabet[i]*num for i, num in enumerate(letterpool) ]
    return ''.join(letterlist)


def letterstopool(letters):
    letterpool = [0 for i in range(26)]
    for l in letters.upper():
        letterindex = alphabet.find(l)
        print(letterindex)
        letterpool[letterindex] += 1
    return letterpool
    
    
def pickletter(letterpool):
    poolstring = ''.join([ alphabet[i]*num for i, num in enumerate(letterpool) ])
    letter = random.choice(poolstring)
    return letter

    
def check_fully_contained(donor, target):
    '''Check if donor is fully contained in target. Return any remaining letters from target'''
    for l in donor:
        index = target.find(l)

        # letter not in target word - abort
        if index < 0:
            return False
        else:
            # remove the letter
            target = target[:index] + target[index+1:]
    
    # return remaining letters from target word
    return target
    
    
def recursive(target, played_words, letterpool):
    target = target.upper()

    if played_words != []:
        for donor in played_words:
             # Check if any of the played words are fully contained in the target word
            remaining = check_fully_contained(donor, target)
            if remaining == False:
                # try the next word in played_words
                continue
            elif remaining == '':
                # found a word to steal - done
                played_words.remove(donor)
                print('stealing ' + donor + ' to make ' + target)
                return True, letterpool, played_words
            else:
                # found a word to steal for some of the letters needed - we need to go deeper
                played_words.remove(donor)
                result, newpool, new_played_words = recursive(remaining, played_words, letterpool)
                print('stealing ' + donor + ' to make ' + target)
                if result == True:
                    return True, newpool, new_played_words
                else:
                    return False, letterpool, played_words
                
    # take a tile from the pool for each letter in the word
    newpool = letterpool.copy()
    target_remaining = ''
    for letter in target:
        letterindex = alphabet.find(letter)
        newpool[letterindex] -= 1

        # did we run out of this letter? add it to the remaining (leftover) letters
        if newpool[letterindex] < 0:
            target_remaining = target_remaining + letter
    print('letters remaining in pool after making', target , ':', pooltoletters(newpool))


    if all(map(lambda x: x>=0, newpool)):
        return True, newpool, played_words
    else:
        return False, letterpool, played_words
    return False, letterpool, played_words
                

In [2]:
text = widgets.Text()
display(text)

def handle(sender):
    print(text.value)
    
text.on_submit(handle)

Text(value='')